In [1]:
import time
import sys
from rfsoc_qsfp_offload.overlay import Overlay

board_ip = '192.168.4.99'
client_ip = '192.168.4.1'

print("Initializing RFSoC QSFP Offload Overlay")
ol = Overlay(ignore_version=True)
ol.cmac.mmio.write(0x107C, 0x3) # RSFEC_CONFIG_ENABLE
ol.cmac.mmio.write(0x1000, 0x7) # RSFEC_CONFIG_INDICATION_CORRECTION

# Wait for overlay to initialize
time.sleep(5)

Starting FM capture at 90.100000MHz
Initializing RFSoC QSFP Offload Overlay


In [2]:
# Start 100G network
ol.cmac.start()
res = ol.netlayer.set_ip_address(board_ip, debug=True)
ol.netlayer.sockets[0] = (client_ip, 60133, 60133, True)
ol.netlayer.populate_socket_table()
print("Network confguration complete IP: %s" % (res['inet addr']))
ol.source_select(1) # Select RF ADC as packet source

Network confguration complete IP: 192.168.4.99


In [7]:
f_c = 98.0                      # MHz
ADC_TILE = 2                    # ADC Tile 226 (ADC A + B)
ADC_BLOCK = 0                   # ADC Block 0 (ADC B)
ADC_SAMPLE_FREQUENCY = 1024     # MSps
ADC_PLL_FREQUENCY    = 491.52   # MHz 
ADC_FC = -1*f_c                 # Tune to center frequency

# Stop packet generator if already started
ol.packet_generator.disable()

# Start ADC
ol.initialise_adc(tile=ADC_TILE,
                    block=ADC_BLOCK,
                    pll_freq=ADC_PLL_FREQUENCY,
                    fs=ADC_SAMPLE_FREQUENCY,
                    fc=ADC_FC)

# Decimate by (16x)
ol.set_decimation(tile=ADC_TILE,block=ADC_BLOCK,sample_rate=64e6)

# Set packet size
ol.packet_generator.packetsize = 128 # 128 * 64 bytes = 8192 bytes to be sent
ol.packet_generator.enable()

print("Starting UDP stream")
        

In [5]:
# Stop packet generator
ol.packet_generator.disable()